In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm
from tqdm import trange
%matplotlib inline

In [ ]:
epochs = np.arange(0, 500, 10)

In [ ]:
def read_list_of_arrays(filename):
    A = pickle.load(open(filename, 'rb'))
    
    dim = A[0].flatten().shape[0]
    B = np.zeros((len(A), dim))

    for i in range(len(A)):
        B[i, :] = A[i].flatten()
        
    return B

In [ ]:
cloud_base = read_list_of_arrays('/gan-clouds/covid_real.pickle')

clouds = []
for ep in epochs:
    clouds.append(read_list_of_arrays('/gan-clouds/covid_generated/covid_generated%d.pickle' % ep))

In [ ]:
cloud_base_negative = read_list_of_arrays('/gan-clouds/covid_real_negative.pickle')

In [ ]:
cloud_base.shape, cloud_base_negative.shape

In [ ]:
for cloud in clouds:
    print(cloud.shape)

### Compute cross-barcodes 

In [ ]:
import mtd

In [ ]:
res1 = []
trials = 50

for i in trange(len(clouds)):
    np.random.seed(7)
    barcs = [mtd.calc_cross_barcodes(cloud_base, clouds[i], batch_size1 = 100, batch_size2 = 1000,\
                                          cuda = 3, pdist_device = 'gpu') for _ in range(trials)]
    res1.append(barcs)

In [ ]:
res2 = []
trials = 50

for i in trange(len(clouds)):
    np.random.seed(7)
    barcs = [mtd.calc_cross_barcodes(clouds[i], cloud_base, batch_size1 = 100, batch_size2 = 1000,\
                                          cuda = 3, pdist_device = 'gpu') for _ in range(trials)]
    res2.append(barcs)

### Compare covid positive vs. covid negative

In [ ]:
sanity1 = []
trials = 50

np.random.seed(7)
sanity1 = [mtd.calc_cross_barcodes(cloud_base, cloud_base_negative, batch_size1 = 100, batch_size2 = 1000,\
                                          cuda = 0, pdist_device = 'gpu') for _ in range(trials)]

In [ ]:
sanity2 = []
trials = 50

np.random.seed(7)
sanity2 = [mtd.calc_cross_barcodes(cloud_base_negative, cloud_base, batch_size1 = 100, batch_size2 = 1000,\
                                          cuda = 0, pdist_device = 'gpu') for _ in range(trials)]

In [ ]:
def get_scores(res, args_dict, trials = 10):

    scores = []

    for i in range(len(res)): 
        barc_list = []
        
        for exp_id, elem in enumerate(res[i]):
            barc_list.append(mtd.get_score(elem, **args_dict))
        
        m = sum(barc_list) / len(barc_list)
        s = np.std(barc_list) / pow(len(barc_list), 0.5)
        
        scores.append(m)

    return scores

In [ ]:
get_scores([sanity1], {'h_idx' : 1, 'kind' : 'sum_length'})

In [ ]:
get_scores([sanity2], {'h_idx' : 1, 'kind' : 'sum_length'})

In [ ]:
scores = get_scores(res1, {'h_idx' : 1, 'kind' : 'sum_length'})

In [ ]:
for ep, s in zip(epochs, scores):
    print(s)

In [ ]:
scores = get_scores(res2, {'h_idx' : 1, 'kind' : 'sum_length'})

In [ ]:
for ep, s in zip(epochs, scores):
    print(s)

In [ ]:
pickle.dump(res1, open('res1_covidgan.pickle', 'wb'))
pickle.dump(res2, open('res2_covidgan.pickle', 'wb'))

### PCA

In [ ]:
from sklearn.decomposition import PCA
%pylab inline

In [ ]:
# Create data
def plot2(data, groups = ("base", "cloud")):
    colors = ("red", "green")

    # Create plot
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    for data, color, group in zip(data, colors, groups):
        x, y = data
        ax.scatter(x, y, alpha=0.5, c=color, edgecolors='none', s=5, label=group)

    #plt.title('Matplot scatter plot')
    plt.legend(loc=2)
    plt.show()

#### PCA from base+last GAN

In [ ]:
all_pca = []

for i in range(len(epochs)):
    pca = PCA(n_components=2)

    cb = np.concatenate((cloud_base, clouds[-1]))
    pca.fit(cb)

    cb = cloud_base
    cloud_base_pca = pca.transform(cb)
    data = [(cloud_base_pca[:,0], cloud_base_pca[:,1])]

    cg = clouds[i]

    cloud_pca = pca.transform(cg)
    data.append((cloud_pca[:,0], cloud_pca[:,1]))
    
    all_pca.append(data)

    plot2(data, groups = ("real", "generated, epoch %d" % epochs[i]))

In [ ]:
#pickle.dump(all_pca, open('covidgan_all_pca.pickle', 'wb'))